In [1]:
from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier(alpha=0.0, fit_intercept=False)

import numpy as np
import graphlearning as gl
import time
from tqdm.notebook import tqdm

labels = gl.datasets.load('mnist', labels_only=True)
W = gl.weightmatrix.knn('mnist', 10, metric='vae')

G = gl.graph(W)
num_train_per_class = 2

In [2]:
lesslaccs = []
psslaccs = []

for _ in tqdm(range(100)):
    vals_rsvd, vecs_rsvd = G.eigen_decomp(normalization='normalized', k=10, method='lowrank', q=50, c=50)

    train_ind = gl.trainsets.generate(labels, rate=num_train_per_class)
    train_labels = labels[train_ind]
    
    b = np.zeros((train_labels.size, train_labels.max() + 1))
    b[np.arange(train_labels.size), train_labels] = 1
    
    # laplacian eigenmaps ssl
    ridge.fit(vecs_rsvd[train_ind], train_labels)
    y_pred = ridge.predict(vecs_rsvd)
    lesslacc = (y_pred == labels).sum() / labels.shape[0]
    
    # procrustes ssl
    u,s,vt = np.linalg.svd(vecs_rsvd[train_ind].T@b)
    coef = u@vt
    reg_embeddings = vecs_rsvd@coef
    
    psslacc = (reg_embeddings.argmax(1) == labels).sum() / labels.shape[0]
    
    lesslaccs.append(lesslacc)
    psslaccs.append(psslacc)
    
print(f'mean Laplacian Eigenmaps-ssl accuracy: {np.mean(lesslaccs):.3f} std: {np.std(lesslaccs):.3f}')
print(f'mean Procrustes-ssl accuracy: {np.mean(psslaccs):.3f} std: {np.std(psslaccs):.3f}')

  0%|          | 0/100 [00:00<?, ?it/s]

mean Laplacian Eigenmaps-ssl accuracy: 0.865 std: 0.035
mean Procrustes-ssl accuracy: 0.894 std: 0.024
